# Step 01: Execute Analysis

This notebook submits analysis jobs to Moody's Risk Modeler.

**Tasks:**
- Retrieve Analysis batch from Stage_01/Step_03
- Review analysis job configurations
- Submit analysis jobs to Moody's API
- Track job completion status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Execute_Analysis.ipynb')

# Display context
ux.header("Execute Analysis Batch")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Analysis Batch

In [ ]:
# Retrieve Analysis batch from Stage_01/Step_03
ux.subheader("Retrieve Analysis Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.ANALYSIS not in batches:
    raise ValueError(f"Analysis batch not found. Available: {list(batches.keys())}")

analysis_batch_id = int(batches[BatchType.ANALYSIS])

ux.success(f"Retrieved Analysis batch: ID={analysis_batch_id}")
step.log(f"Retrieved Analysis batch: ID={analysis_batch_id}")

# Validate batch before proceeding
ux.subheader("Validate Batch")
validation_errors = validate_batch(analysis_batch_id)

if validation_errors:
    ux.error("✗ Batch validation failed:")
    for error in validation_errors:
        ux.error(f"  {error}")
    step.log(f"Batch validation failed: {len(validation_errors)} error(s)")
    
    # Mark step as failed
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message="\n".join(validation_errors))
    
    raise SystemExit("Batch validation failed - see errors above")
else:
    ux.success("✓ Batch validation passed")

## 3) Reconcile Batch State

Check if any analyses already exist in Moody's for the jobs in this batch.

In [ ]:
# Reconcile batch state with Moody's
from helpers.batch import reconcile_analysis_batch

ux.subheader("Reconcile Batch State")

# Action constants
ACTION_SKIP = 'skip'
ACTION_FRESH = 'fresh'
ACTION_RESUBMIT = 'resubmit'
ACTION_RESUBMIT_FAILED = 'resubmit_failed'
ACTION_RESUBMIT_NO_ANALYSIS = 'resubmit_no_analysis'
ACTION_SUBMIT_ALL = 'submit_all'
ACTION_SUBMIT_BLOCKED = 'submit_blocked'

irp_client = IRPClient()
recon = reconcile_analysis_batch(analysis_batch_id, irp_client)

# Extract categories
jobs_successful = recon['jobs_successful']
jobs_failed = recon['jobs_failed']
jobs_missing_analysis = recon['jobs_missing_analysis']
jobs_pending = recon['jobs_pending']
jobs_fresh = recon['jobs_fresh']
jobs_blocked = recon['jobs_blocked']
existing_analyses = recon['existing_analyses']

# Store for later use
RESUBMIT_ACTION = None
JOBS_TO_RESUBMIT = []
JOBS_TO_SUBMIT = []
JOBS_TO_DELETE_ANALYSIS = []

# --- Display Summary ---
ux.info(f"Batch ID: {recon['batch_id']}  |  Status: {recon['batch_status']}  |  Total Jobs: {recon['total_jobs']}")

# Build a compact status line
status_parts = []
if jobs_successful:
    status_parts.append(f"{len(jobs_successful)} successful")
if jobs_failed:
    status_parts.append(f"{len(jobs_failed)} failed")
if jobs_missing_analysis:
    status_parts.append(f"{len(jobs_missing_analysis)} missing analysis")
if jobs_pending:
    status_parts.append(f"{len(jobs_pending)} pending")
if jobs_fresh:
    status_parts.append(f"{len(jobs_fresh)} ready")
if jobs_blocked:
    status_parts.append(f"{len(jobs_blocked)} blocked")

if status_parts:
    ux.info(f"Jobs: {', '.join(status_parts)}")

# --- Determine Action ---
if len(jobs_fresh) == recon['total_jobs']:
    ux.success("\nAll jobs ready for initial submission.")
    RESUBMIT_ACTION = ACTION_FRESH

elif len(jobs_fresh) > 0 and not jobs_failed and not jobs_missing_analysis and not jobs_successful and not jobs_blocked:
    ux.success(f"\n{len(jobs_fresh)} job(s) ready for initial submission.")
    RESUBMIT_ACTION = ACTION_FRESH

elif jobs_pending and not jobs_failed and not jobs_missing_analysis and not jobs_fresh and not jobs_blocked:
    ux.info("\nAll jobs are either successful or still processing. Nothing to submit.")
    RESUBMIT_ACTION = ACTION_SKIP

elif jobs_failed or jobs_missing_analysis or jobs_successful or jobs_blocked:
    # Show issues that need attention
    if jobs_failed:
        ux.error(f"\nFailed ({len(jobs_failed)}):")
        for job in jobs_failed:
            ux.error(f"  {job['analysis_name']} - {job['status']}")
    
    if jobs_missing_analysis:
        ux.warning(f"\nMissing Analysis ({len(jobs_missing_analysis)}):")
        for job in jobs_missing_analysis:
            ux.warning(f"  {job['analysis_name']} - job finished but analysis not found")
    
    if jobs_blocked:
        ux.warning(f"\nBlocked ({len(jobs_blocked)}):")
        for job in jobs_blocked:
            ux.warning(f"  {job['analysis_name']} - analysis already exists, not yet submitted")
    
    if jobs_successful:
        ux.success(f"\nSuccessful ({len(jobs_successful)}):")
        for job in jobs_successful:
            ux.success(f"  {job['analysis_name']}")
    
    # Show fresh jobs that will be submitted alongside any resubmission
    if jobs_fresh:
        ux.info(f"\nReady for submission ({len(jobs_fresh)}):")
        for job in jobs_fresh:
            ux.info(f"  {job['analysis_name']}")
    
    # Calculate groups
    jobs_without_analysis = jobs_failed + jobs_missing_analysis
    jobs_with_existing_analysis = jobs_successful + jobs_blocked
    all_actionable_jobs = jobs_failed + jobs_missing_analysis + jobs_successful + jobs_blocked
    all_need_action = len(jobs_successful) == 0 and len(all_actionable_jobs) == recon['total_jobs'] - len(jobs_pending) - len(jobs_fresh)
    
    # Build options
    ux.info("\n" + "-"*40)
    option_num = 1
    options_map = {}
    
    # Note about fresh jobs if they exist
    fresh_note = f" + submit {len(jobs_fresh)} ready" if jobs_fresh else ""
    
    if all_need_action and all_actionable_jobs:
        # Single option when all jobs need action
        if jobs_with_existing_analysis:
            ux.info(f"  {option_num}. Resubmit all ({len(all_actionable_jobs)} jobs, delete {len(jobs_with_existing_analysis)} analyses){fresh_note}")
            options_map[str(option_num)] = ACTION_SUBMIT_ALL
        else:
            ux.info(f"  {option_num}. Resubmit all ({len(all_actionable_jobs)} jobs){fresh_note}")
            options_map[str(option_num)] = ACTION_RESUBMIT
        option_num += 1
    else:
        # Granular options
        if jobs_failed:
            ux.info(f"  {option_num}. Resubmit failed only ({len(jobs_failed)} jobs){fresh_note}")
            options_map[str(option_num)] = ACTION_RESUBMIT_FAILED
            option_num += 1
        
        if jobs_without_analysis and len(jobs_without_analysis) != len(jobs_failed):
            ux.info(f"  {option_num}. Resubmit all without analysis ({len(jobs_without_analysis)} jobs){fresh_note}")
            options_map[str(option_num)] = ACTION_RESUBMIT_NO_ANALYSIS
            option_num += 1
        
        if jobs_with_existing_analysis:
            ux.info(f"  {option_num}. Resubmit ALL ({len(all_actionable_jobs)} jobs, delete {len(jobs_with_existing_analysis)} analyses){fresh_note}")
            options_map[str(option_num)] = ACTION_SUBMIT_ALL
            option_num += 1
        
        if jobs_blocked and not jobs_failed and not jobs_missing_analysis and not jobs_successful:
            ux.info(f"  {option_num}. Submit blocked ({len(jobs_blocked)} jobs, delete analyses first){fresh_note}")
            options_map[str(option_num)] = ACTION_SUBMIT_BLOCKED
            option_num += 1
    
    ux.info(f"  {option_num}. Skip")
    options_map[str(option_num)] = ACTION_SKIP
    
    choice = input(f"\nSelect option (1-{option_num}): ").strip()
    selected_action = options_map.get(choice, ACTION_SKIP)
    
    # Prepare jobs and determine what will happen
    jobs_to_resubmit = []  # Jobs that need resubmission (terminal state)
    jobs_to_submit = []    # Jobs that need first submission (blocked)
    
    if selected_action == ACTION_RESUBMIT_FAILED:
        jobs_to_resubmit = jobs_failed
        JOBS_TO_RESUBMIT = [j['job_id'] for j in jobs_to_resubmit]
    elif selected_action == ACTION_RESUBMIT_NO_ANALYSIS:
        jobs_to_resubmit = jobs_without_analysis
        JOBS_TO_RESUBMIT = [j['job_id'] for j in jobs_to_resubmit]
    elif selected_action == ACTION_RESUBMIT:
        jobs_to_resubmit = all_actionable_jobs
        JOBS_TO_RESUBMIT = [j['job_id'] for j in jobs_to_resubmit]
    elif selected_action == ACTION_SUBMIT_ALL:
        jobs_to_resubmit = jobs_failed + jobs_missing_analysis + jobs_successful
        jobs_to_submit = jobs_blocked
        JOBS_TO_RESUBMIT = [j['job_id'] for j in jobs_to_resubmit]
        JOBS_TO_SUBMIT = [j['job_id'] for j in jobs_to_submit]
        JOBS_TO_DELETE_ANALYSIS = jobs_successful + jobs_blocked
    elif selected_action == ACTION_SUBMIT_BLOCKED:
        jobs_to_submit = jobs_blocked
        JOBS_TO_SUBMIT = [j['job_id'] for j in jobs_to_submit]
        JOBS_TO_DELETE_ANALYSIS = jobs_blocked
    else:
        selected_action = ACTION_SKIP
    
    # Confirm if not skipping
    if selected_action != ACTION_SKIP:
        # Show jobs to resubmit (terminal jobs)
        if jobs_to_resubmit:
            ux.info(f"\nJobs to resubmit ({len(jobs_to_resubmit)}):")
            for job in jobs_to_resubmit:
                ux.info(f"  - {job['analysis_name']} (EDM: {job['edm_name']})")
        
        # Show jobs to submit (blocked jobs - first submission)
        if jobs_to_submit:
            ux.info(f"\nBlocked jobs to submit ({len(jobs_to_submit)}):")
            for job in jobs_to_submit:
                ux.info(f"  - {job['analysis_name']} (EDM: {job['edm_name']})")
        
        # Show fresh jobs that will also be submitted
        if jobs_fresh:
            ux.info(f"\nFresh jobs to submit ({len(jobs_fresh)}):")
            for job in jobs_fresh:
                ux.info(f"  - {job['analysis_name']} (EDM: {job['edm_name']})")
        
        if JOBS_TO_DELETE_ANALYSIS:
            ux.warning(f"\n{len(JOBS_TO_DELETE_ANALYSIS)} analysis(es) will be deleted first.")
        
        confirm = input("\nProceed? (y/n): ").strip().lower()
        if confirm in ['yes', 'y', 'Y']:
            # Normalize action to what submission cell expects
            if selected_action in [ACTION_RESUBMIT_FAILED, ACTION_RESUBMIT_NO_ANALYSIS, ACTION_RESUBMIT]:
                RESUBMIT_ACTION = ACTION_RESUBMIT
            else:
                RESUBMIT_ACTION = selected_action
            ux.success("Confirmed.")
        else:
            RESUBMIT_ACTION = ACTION_SKIP
            JOBS_TO_RESUBMIT = []
            JOBS_TO_SUBMIT = []
            JOBS_TO_DELETE_ANALYSIS = []
            ux.info("Cancelled.")
    else:
        RESUBMIT_ACTION = ACTION_SKIP
        ux.info("\nSkipped.")

else:
    ux.info("\nNo jobs available for submission.")
    RESUBMIT_ACTION = ACTION_SKIP

step.log(f"Reconciliation: action={RESUBMIT_ACTION}, resubmit={len(JOBS_TO_RESUBMIT)}, submit={len(JOBS_TO_SUBMIT)}, fresh={len(jobs_fresh)}")

## 4) Submit Analysis Batch to Moody's

In [ ]:
# Submit batch based on reconciliation action
from helpers.job import resubmit_jobs, delete_analyses_for_jobs

ux.subheader("Submit Batch to Moody's")

deletion_errors = []
failed_count = 0

if RESUBMIT_ACTION == ACTION_SKIP:
    ux.info("Submission skipped by user.")
    result = {'submitted_jobs': 0, 'batch_status': recon['batch_status'], 'jobs': []}

elif RESUBMIT_ACTION == ACTION_FRESH:
    # Fresh submission - use normal submit_batch
    ux.info("Submitting fresh batch...")
    result = submit_batch(analysis_batch_id, irp_client, step_id=step.step_id)
    failed_count = len([j for j in result['jobs'] if 'error' in j])

    ux.success(f"\nBatch submission completed")
    ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
    ux.info(f"  Status: {result['batch_status']}")

elif RESUBMIT_ACTION == ACTION_RESUBMIT:
    # Delete existing analyses first (only for jobs that have them)
    if JOBS_TO_DELETE_ANALYSIS:
        ux.info(f"Deleting {len(JOBS_TO_DELETE_ANALYSIS)} existing analyses...")
        deletion_errors = delete_analyses_for_jobs(JOBS_TO_DELETE_ANALYSIS, existing_analyses, irp_client)
        for job in JOBS_TO_DELETE_ANALYSIS:
            if not any(job['analysis_name'] in err for err in deletion_errors):
                ux.info(f"  Deleted: {job['analysis_name']}")
        for error in deletion_errors:
            ux.error(f"  Failed: {error}")

    # Check if any deletions failed
    if deletion_errors:
        ux.error(f"\n{len(deletion_errors)} analysis deletion(s) failed. Cannot proceed with resubmission.")
        ux.info("\nPlease manually delete the following analyses in Moody's before retrying:")
        for error in deletion_errors:
            ux.error(f"  - {error}")

        result = {'submitted_jobs': 0, 'batch_status': recon['batch_status'], 'jobs': []}
    else:
        # All deletions succeeded (or none needed) - now resubmit jobs
        ux.info(f"\nResubmitting {len(JOBS_TO_RESUBMIT)} jobs...")
        resubmit_result = resubmit_jobs(JOBS_TO_RESUBMIT, irp_client, BatchType.ANALYSIS)

        result = {
            'submitted_jobs': resubmit_result['success_count'],
            'batch_status': 'ACTIVE',
            'jobs': resubmit_result['successful'] + [{'job_id': f['job_id'], 'error': f['error']} for f in resubmit_result['failed']]
        }
        failed_count = resubmit_result['failure_count']

        ux.success(f"\nResubmission completed")
        ux.info(f"  Submitted: {resubmit_result['success_count']} jobs")
        ux.info(f"  Failed: {resubmit_result['failure_count']} jobs")

elif RESUBMIT_ACTION == ACTION_SUBMIT_ALL:
    # Delete existing analyses first, then resubmit terminal jobs and submit blocked jobs
    if JOBS_TO_DELETE_ANALYSIS:
        ux.info(f"Deleting {len(JOBS_TO_DELETE_ANALYSIS)} existing analyses...")
        deletion_errors = delete_analyses_for_jobs(JOBS_TO_DELETE_ANALYSIS, existing_analyses, irp_client)
        for job in JOBS_TO_DELETE_ANALYSIS:
            if not any(job['analysis_name'] in err for err in deletion_errors):
                ux.info(f"  Deleted: {job['analysis_name']}")
        for error in deletion_errors:
            ux.error(f"  Failed: {error}")

    if deletion_errors:
        ux.error(f"\n{len(deletion_errors)} analysis deletion(s) failed. Cannot proceed.")
        ux.info("\nPlease manually delete the following analyses in Moody's before retrying:")
        for error in deletion_errors:
            ux.error(f"  - {error}")

        result = {'submitted_jobs': 0, 'batch_status': recon['batch_status'], 'jobs': []}
    else:
        all_jobs = []
        total_submitted = 0
        total_failed = 0

        # Resubmit terminal jobs (failed, missing_analysis, successful)
        if JOBS_TO_RESUBMIT:
            ux.info(f"\nResubmitting {len(JOBS_TO_RESUBMIT)} terminal jobs...")
            resubmit_result = resubmit_jobs(JOBS_TO_RESUBMIT, irp_client, BatchType.ANALYSIS)
            total_submitted += resubmit_result['success_count']
            total_failed += resubmit_result['failure_count']
            all_jobs.extend(resubmit_result['successful'])
            all_jobs.extend([{'job_id': f['job_id'], 'error': f['error']} for f in resubmit_result['failed']])

            ux.info(f"  Resubmitted: {resubmit_result['success_count']}, Failed: {resubmit_result['failure_count']}")

        # Submit blocked jobs (INITIATED jobs with now-deleted analyses) using submit_batch
        if JOBS_TO_SUBMIT:
            ux.info(f"\nSubmitting {len(JOBS_TO_SUBMIT)} blocked jobs...")
            # submit_batch will only submit INITIATED jobs, which is exactly what we have
            submit_result = submit_batch(analysis_batch_id, irp_client, step_id=step.step_id)
            submit_success = submit_result['submitted_jobs']
            submit_failed = len([j for j in submit_result['jobs'] if 'error' in j])
            
            total_submitted += submit_success
            total_failed += submit_failed
            all_jobs.extend(submit_result['jobs'])
            ux.info(f"  Submitted: {submit_success}, Failed: {submit_failed}")

        result = {
            'submitted_jobs': total_submitted,
            'batch_status': 'ACTIVE',
            'jobs': all_jobs
        }
        failed_count = total_failed

        ux.success(f"\nSubmission completed")
        ux.info(f"  Total submitted: {total_submitted} jobs")
        ux.info(f"  Total failed: {total_failed} jobs")

elif RESUBMIT_ACTION == ACTION_SUBMIT_BLOCKED:
    # Delete existing analyses for blocked jobs, then submit them using submit_batch
    if JOBS_TO_DELETE_ANALYSIS:
        ux.info(f"Deleting {len(JOBS_TO_DELETE_ANALYSIS)} existing analyses...")
        deletion_errors = delete_analyses_for_jobs(JOBS_TO_DELETE_ANALYSIS, existing_analyses, irp_client)
        for job in JOBS_TO_DELETE_ANALYSIS:
            if not any(job['analysis_name'] in err for err in deletion_errors):
                ux.info(f"  Deleted: {job['analysis_name']}")
        for error in deletion_errors:
            ux.error(f"  Failed: {error}")

    if deletion_errors:
        ux.error(f"\n{len(deletion_errors)} analysis deletion(s) failed. Cannot proceed.")
        ux.info("\nPlease manually delete the following analyses in Moody's before retrying:")
        for error in deletion_errors:
            ux.error(f"  - {error}")

        result = {'submitted_jobs': 0, 'batch_status': recon['batch_status'], 'jobs': []}
    else:
        # Submit blocked jobs using submit_batch (they are INITIATED, so submit_batch handles them)
        ux.info(f"\nSubmitting {len(JOBS_TO_SUBMIT)} blocked jobs...")
        result = submit_batch(analysis_batch_id, irp_client, step_id=step.step_id)
        failed_count = len([j for j in result['jobs'] if 'error' in j])

        ux.success(f"\nSubmission completed")
        ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
        ux.info(f"  Failed: {failed_count} jobs")

# Check for errors
if failed_count > 0:
    ux.warning(f"\n{failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data
output_data = {
    'batch_id': analysis_batch_id,
    'batch_type': BatchType.ANALYSIS,
    'batch_status': result['batch_status'],
    'submitted_jobs': result['submitted_jobs'],
    'failed_jobs': failed_count,
    'deletion_errors': len(deletion_errors),
    'action': RESUBMIT_ACTION if RESUBMIT_ACTION != 'fresh' else 'submit'
}

# Check if any deletions failed
if deletion_errors:
    error_message = f"{len(deletion_errors)} analysis deletion(s) failed:\n" + "\n".join(deletion_errors)
    
    # Mark step as failed in database
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("ANALYSIS DELETION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {analysis_batch_id}")
    ux.error(f"Failed deletions: {len(deletion_errors)}")
    ux.info("\nThe following analyses could not be deleted:")
    for error in deletion_errors:
        ux.error(f"  - {error}")
    ux.info("\nPlease manually delete these analyses in Moody's Risk Modeler,")
    ux.info("then re-run this notebook to retry submission.")

# Check if any jobs failed to submit
elif failed_count > 0:
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}"
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)

    # Note: Teams notification already sent from batch.py for each failed job
    # Mark step as failed in database (skip duplicate notification)
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)

    ux.error("\n" + "="*60)
    ux.error("BATCH SUBMISSION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {analysis_batch_id}")
    ux.info(f"Submitted: {result['submitted_jobs']} job(s)")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nFailed jobs:")
    for error in failed_job_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease review the errors and resubmit failed jobs.")
else:
    # Complete the step successfully (includes skip case)
    step.complete(output_data)

    ux.success("\n" + "="*60)
    if RESUBMIT_ACTION == 'skip':
        ux.success("STEP COMPLETED - SUBMISSION SKIPPED")
    else:
        ux.success("ANALYSIS BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nBatch ID: {analysis_batch_id}")
    ux.info(f"Submitted {result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Monitor job progress in Step_02 or proceed to Grouping stage")